In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from data_import import (
    load_data, load_ecb_1y_yield,
    fill_liabilities, drop_high_leverage_firms,
    prepare_merton_inputs
)

print(Path.cwd())

c:\Users\afons\OneDrive\Desktop\ESE\FCS\Merton_NIGbayesian


In [2]:
# data loading and initial processing
ret_daily, bs, coverage = load_data(
    Path.cwd() / "Jan2025_Accenture_Dataset_ErasmusCase.xlsx",
    start_date="2012-01-01",
    end_date="2025-12-19",
    enforce_coverage=True,
    coverage_tol=0.95,
    liabilities_scale="auto",
    verbose=True,
)

df_rf = load_ecb_1y_yield(
    startPeriod="2010-01-01",
    endPeriod="2025-12-31",
    out_file="ecb_yc_1y_aaa.xml",
    verify_ssl=True,  # recommended if it works
)

df_cal = ret_daily[["date"]].drop_duplicates().sort_values("date").reset_index(drop=True)

debt_daily = fill_liabilities(bs, df_cal)

ret_filt, bs_filt, lev_by_firm, dropped = drop_high_leverage_firms(
    ret_daily,
    bs,
    df_calendar=df_cal,
    debt_daily=debt_daily,
    lev_threshold=8.0,
    lev_agg="median",
    verbose=True,
)

# keep debt panel consistent with filtered firms
keep = set(ret_filt["gvkey"].astype(str).unique())
debt_daily_filt = debt_daily[debt_daily["gvkey"].astype(str).isin(keep)].copy()

# Merton
merton_df = prepare_merton_inputs(ret_filt, bs_filt, df_rf, debt_daily=debt_daily_filt)

[load_data] Firms (ret_daily): 46
[load_data] Date range (ret_daily): 2012-01-03 .. 2025-12-19
[load_data] Coverage min/median/max: 0.999 / 1.000 / 1.000
[load_data] liabilities_scale_used: 1e+06
[load_data] QA mcap_reported<=0 rows (raw windowed mkt): 62
Data has been written to ecb_yc_1y_aaa.xml
[drop_high_leverage_firms] agg=median, threshold=8.0
[drop_high_leverage_firms] firms before: 46 | after: 36
[drop_high_leverage_firms] dropped firms: 10


In [3]:
# Check missingness of sigma_E
print("Missing sigma_E %:", merton_df["sigma_E"].isna().mean() * 100)
print(merton_df["sigma_E"].describe())

# Check missingness of B and r
print("Missing B %:", merton_df["B"].isna().mean() * 100)
print("Missing r %:", merton_df["r"].isna().mean() * 100)

Missing sigma_E %: 3.431054858754908
count    126655.000000
mean          0.254476
std           0.083942
min           0.117892
25%           0.195147
50%           0.238274
75%           0.292771
max           0.743236
Name: sigma_E, dtype: float64
Missing B %: 0.0
Missing r %: 0.0


In [4]:
# BUILDING THE CALIBRATION DATASET DROPPING ROWS WITH MISSING INPUTS
df = merton_df.copy()

# first date where B becomes available for each firm
first_B_date = (
    df.dropna(subset=["B"])
      .groupby("gvkey")["date"]
      .min()
      .rename("first_B_date")
)
# first date where sigma_E becomes available for each firm
first_sigma_date = (
    df.dropna(subset=["sigma_E"])
      .groupby("gvkey")["date"]
      .min()
      .rename("first_sigma_date")
)

starts = pd.concat([first_B_date, first_sigma_date], axis=1)
starts["calib_start"] = starts[["first_B_date","first_sigma_date"]].max(axis=1)

# attach and filter
df2 = df.merge(starts["calib_start"], on="gvkey", how="left")

calib = (
    df2[df2["date"] >= df2["calib_start"]]
      .dropna(subset=["E","B","r","sigma_E"])
      .query("E > 0 and B > 0")
      .copy()
      .rename(columns={"B":"B_drop"})
)

print("Rows before:", len(df), "Rows after firm-specific start + required inputs:", len(calib))
print("Dropped %:", (len(df)-len(calib))/len(df))
print("Missing values in calibration dataset:")
print(calib.isna().sum())

calib_drop = calib.copy()

Rows before: 131155 Rows after firm-specific start + required inputs: 126655
Dropped %: 0.03431054858754908
Missing values in calibration dataset:
gvkey            0
date             0
E                0
logret_mcap      0
isin             0
company          0
country_iso      0
r                0
B_drop           0
sigma_E_daily    0
sigma_E          0
calib_start      0
dtype: int64


In [ ]:
# 5) Rolling settings + one-time preprocessing

from merton_model_afonso import (
    calibrate_sigmaV_window_weekly_merton,
    invert_asset_one_week_merton,
)

from merton_pd_afonso import (
    estimate_mu_from_weekly_implied_assets,
    merton_pd_rn_1y,
    merton_pd_physical_1y,
)

# ----------------------------
# Rolling configuration
# ----------------------------
TRAIN_YEARS = 2
STEP_FREQ = "QE"               # re-estimate every quarter
WEEK_ENDING = "W-FRI"         # weekly points ending on Friday (but we use last trading day in that week)
T_HORIZON = 1.0               # KMV-style horizon for DD/PD
DATA_END = pd.Timestamp("2024-12-31")  # stop at end of dataset
MIN_DAILY_ROWS = 10

# OOS definition: "next quarter"
# We'll run windows where the OOS quarter end is <= DATA_END.
# So the last train_end will be the quarter end immediately before DATA_END.
LAST_TRAIN_END = (DATA_END - pd.offsets.QuarterEnd(1))  # e.g., 2024-09-30

# Debug / runtime control: start small, then scale up.
MAX_FIRMS = 2   # e.g. 2 for quick test; None for all firms
MAX_WINDOWS = None # e.g. 2 for quick test; None for all windows


# ----------------------------
# One-time preprocessing for speed
# ----------------------------
panel = merton_df.copy()
panel["gvkey"] = panel["gvkey"].astype(str)
panel["date"] = pd.to_datetime(panel["date"])

# Keep only columns we need repeatedly (reduces memory + speeds groupby slicing)
needed_cols = ["gvkey","date","company","E","B","r","sigma_E"]
panel = panel[[c for c in needed_cols if c in panel.columns]].copy()

# Coerce numeric once (avoid repeating inside loops)
for c in ["E","B","r","sigma_E"]:
    if c in panel.columns:
        panel[c] = pd.to_numeric(panel[c], errors="coerce")

# Add constants once
panel["T"] = float(T_HORIZON)

# Basic cleaning (same spirit as your estimation cell)
panel = (
    panel.dropna(subset=["date","E","B","r","T"])
         .query("E > 0 and B > 0")
         .sort_values(["gvkey","date"])
)

# Build per-firm daily panels once (fast slicing later)
firm_daily = {}
for gvkey, g in panel.groupby("gvkey", sort=False):
    g = g.sort_values("date")
    # dedupe dates if needed
    g = g.groupby("date", as_index=False).last()
    firm_daily[gvkey] = g.set_index("date")

gvkeys_all = sorted(firm_daily.keys())
if MAX_FIRMS is not None:
    gvkeys_all = gvkeys_all[:int(MAX_FIRMS)]

print("Firms loaded:", len(firm_daily), "| Firms in run:", len(gvkeys_all))
print("Panel date range:", panel["date"].min().date(), "to", panel["date"].max().date())
print("LAST_TRAIN_END:", LAST_TRAIN_END.date(), "| DATA_END:", DATA_END.date())

Firms loaded: 36 | Firms in run: 2
Panel date range: 2012-01-03 to 2025-12-19
LAST_TRAIN_END: 2024-09-30 | DATA_END: 2024-12-31


In [18]:
# 6) Build the rolling quarter schedule

global_min_date = panel["date"].min()

# earliest possible train_end is >= (min_date + TRAIN_YEARS - 1 day), aligned to quarter-ends
earliest_end = (global_min_date + pd.DateOffset(years=TRAIN_YEARS) - pd.Timedelta(days=1))

train_ends = pd.date_range(start=earliest_end, end=LAST_TRAIN_END, freq=STEP_FREQ)
train_ends = pd.to_datetime(train_ends)

if MAX_WINDOWS is not None:
    train_ends = train_ends[:int(MAX_WINDOWS)]

windows = []
for train_end in train_ends:
    train_start = train_end - pd.DateOffset(years=TRAIN_YEARS) + pd.Timedelta(days=1)
    oos_start = train_end + pd.Timedelta(days=1)
    oos_end = train_end + pd.offsets.QuarterEnd(1)  # next quarter end

    windows.append({
        "train_start": pd.Timestamp(train_start),
        "train_end": pd.Timestamp(train_end),
        "oos_start": pd.Timestamp(oos_start),
        "oos_end": pd.Timestamp(oos_end),
    })

windows_df = pd.DataFrame(windows)
windows_df

,train_start,train_end,oos_start,oos_end
0,2012-04-01,2014-03-31,2014-04-01,2014-06-30
1,2012-07-01,2014-06-30,2014-07-01,2014-09-30
2,2012-10-01,2014-09-30,2014-10-01,2014-12-31
3,2013-01-01,2014-12-31,2015-01-01,2015-03-31
4,2013-04-01,2015-03-31,2015-04-01,2015-06-30
5,2013-07-01,2015-06-30,2015-07-01,2015-09-30
6,2013-10-01,2015-09-30,2015-10-01,2015-12-31
7,2014-01-01,2015-12-31,2016-01-01,2016-03-31
8,2014-04-01,2016-03-31,2016-04-01,2016-06-30
9,2014-07-01,2016-06-30,2016-07-01,2016-09-30


In [19]:
# 7) Rolling estimation + PDs + OOS inversion

from tqdm import tqdm

def last_trading_day_each_week(df_daily_indexed, week_ending="W-FRI"):
    """
    df_daily_indexed: daily df with DatetimeIndex (trading days)
    Returns a sorted list of actual trading dates representing the last trading day in each W-FRI week.
    """
    if df_daily_indexed.empty:
        return []
    tmp = df_daily_indexed.reset_index().rename(columns={"index": "date"})
    tmp["week"] = tmp["date"].dt.to_period(week_ending)
    wk = tmp.groupby("week")["date"].max().sort_values()
    return wk.tolist()


# Rolling outputs
roll_summary_rows = []      # firm x window (sigma, mu, status)
roll_weekly_is_rows = []    # weekly in-sample (optional but useful)
roll_weekly_oos_rows = []   # weekly OOS

# Warm-start caches
prev_sigma_init = {}  # gvkey -> last sigma_hat
prev_lastV = {}       # gvkey -> last in-sample V_hat

for w in tqdm(windows, desc="Rolling windows"):
    train_start = w["train_start"]
    train_end   = w["train_end"]
    oos_start   = w["oos_start"]
    oos_end     = w["oos_end"]

    for gvkey in gvkeys_all:
        g_all = firm_daily.get(gvkey)
        if g_all is None or g_all.empty:
            roll_summary_rows.append({
                "gvkey": gvkey, "train_end": train_end, "ok": False, "msg": "missing_firm_panel"
            })
            continue

        # ----------------------------
        # Training slice
        # ----------------------------
        g_train = g_all.loc[(g_all.index >= train_start) & (g_all.index <= train_end)].copy()
        if g_train.empty or len(g_train) < MIN_DAILY_ROWS:
            roll_summary_rows.append({
                "gvkey": gvkey, "train_start": train_start, "train_end": train_end,
                "ok": False, "msg": "too_few_daily_rows_train", "n_daily": int(len(g_train))
            })
            continue

        # Add required columns for calibration function (it expects "date" column, not index)
        g_train2 = g_train.reset_index().rename(columns={"index": "date"})
        g_train2["T"] = float(T_HORIZON)

        # columns are already numeric from preprocessing; keep consistent names
        g_train2 = (
            g_train2.dropna(subset=["date","E","B","r","T"])
                    .query("E > 0 and B > 0")
                    .sort_values("date")
        )

        if len(g_train2) < MIN_DAILY_ROWS:
            roll_summary_rows.append({
                "gvkey": gvkey, "train_start": train_start, "train_end": train_end,
                "ok": False, "msg": "too_few_daily_rows_train_after_clean", "n_daily": int(len(g_train2))
            })
            continue

        # Liability for PD calculation: last available daily point in training window (your approach)
        B_end = float(g_train2["B"].iloc[-1])

        # Warm start sigma
        sigma_init = prev_sigma_init.get(gvkey, None)

        # ----------------------------
        # Estimate sigma_V + weekly implied assets in-sample
        # ----------------------------
        sigmaV_hat, weekly_df, ok, msg = calibrate_sigmaV_window_weekly_merton(
            g_train2,
            week_ending=WEEK_ENDING,
            ann_factor=52.0,
            sigmaV_init=sigma_init,
            E_col="E",
            B_col="B",
            r_col="r",
            T_col="T",
            sigmaE_col="sigma_E",
        )

        if (not ok) or (weekly_df is None) or (len(weekly_df) == 0) or (not np.isfinite(sigmaV_hat)) or (sigmaV_hat <= 0):
            roll_summary_rows.append({
                "gvkey": gvkey, "train_start": train_start, "train_end": train_end,
                "ok": False, "msg": f"calibration_failed:{msg}", "sigma_hat": sigmaV_hat
            })
            continue

        # store for warm-start next window
        prev_sigma_init[gvkey] = float(sigmaV_hat)

        # ----------------------------
        # In-sample mu and PD at last weekly point
        # ----------------------------
        w_is = weekly_df.copy()
        # expected cols: date, V_hat, dlogV, r (based on your earlier usage)
        w_is["date"] = pd.to_datetime(w_is["date"])
        w_is = w_is.sort_values("date")

        # estimate mu from weekly implied assets
        mu_hat = estimate_mu_from_weekly_implied_assets(
            pd.DataFrame({"dlogV": w_is["dlogV"].to_numpy(dtype=float)}),
            float(sigmaV_hat),
            ann_factor=52.0
        )
        mu_hat = float(mu_hat) if np.isfinite(mu_hat) else np.nan

        pd_date = pd.Timestamp(w_is["date"].iloc[-1])
        V_pd = float(w_is["V_hat"].iloc[-1])
        r_pd = float(w_is["r"].iloc[-1])

        PD_Q_1y_is = float(merton_pd_rn_1y(V_pd, B_end, r_pd, float(sigmaV_hat)))
        PD_P_1y_is = float(merton_pd_physical_1y(V_pd, B_end, mu_hat, float(sigmaV_hat))) if np.isfinite(mu_hat) else np.nan

        # store last in-sample V for OOS warm start
        prev_lastV[gvkey] = float(V_pd)

        roll_summary_rows.append({
            "gvkey": gvkey,
            "train_start": train_start, "train_end": train_end,
            "oos_start": oos_start, "oos_end": oos_end,
            "ok": True, "msg": "ok",
            "sigma_hat": float(sigmaV_hat),
            "mu_hat_train": mu_hat,
            "B_end": B_end,
            "pd_date_is": pd_date,
            "PD_Q_1y_is": PD_Q_1y_is,
            "PD_P_1y_is": PD_P_1y_is,
            "n_daily_train": int(len(g_train2)),
            "n_weekly_train": int(len(w_is)),
        })

        # Optional: store weekly in-sample rows (can be useful for diagnostics)
        w_store = w_is[["date","V_hat","dlogV","r"]].copy()
        w_store["gvkey"] = gvkey
        w_store["train_end"] = train_end
        w_store["sigma_hat"] = float(sigmaV_hat)
        w_store["mu_hat_train"] = mu_hat
        w_store["B_end"] = B_end
        roll_weekly_is_rows.append(w_store)

        # ----------------------------
        # OOS slice + weekly inversion for next quarter
        # ----------------------------
        g_oos = g_all.loc[(g_all.index >= oos_start) & (g_all.index <= oos_end)].copy()
        if g_oos.empty:
            continue

        # ensure required cols exist (preprocessing should have done this)
        g_oos = g_oos.dropna(subset=["E","B","r"]).query("E>0 and B>0").copy()
        if g_oos.empty:
            continue

        weekly_dates = last_trading_day_each_week(g_oos, week_ending=WEEK_ENDING)
        if len(weekly_dates) == 0:
            continue

        # invert week by week
        V_prev = prev_lastV.get(gvkey, None)
        V_prev = float(V_prev) if (V_prev is not None and np.isfinite(V_prev) and V_prev > 0) else None

        oos_rows_this = []
        V_prev_for_dlog = V_prev

        # fast lookup by date
        g_oos_idx = g_oos.groupby(g_oos.index).last()

        for d in weekly_dates:
            if d not in g_oos_idx.index:
                continue
            row = g_oos_idx.loc[d]

            E_obs = float(row["E"])
            B_obs = float(row["B"])
            r_obs = float(row["r"])

            V_hat, d1, d2 = invert_asset_one_week_merton(
                E_obs, B_obs, float(r_obs), float(T_HORIZON), float(sigmaV_hat),
                V_prev=V_prev,
                tol=1e-6,
                maxiter=200
            )

            # compute weekly log-return on implied assets (if we have a previous)
            dlogV = np.nan
            if V_prev_for_dlog is not None and np.isfinite(V_prev_for_dlog) and V_prev_for_dlog > 0 and np.isfinite(V_hat) and V_hat > 0:
                dlogV = float(np.log(V_hat / V_prev_for_dlog))

            V_prev_for_dlog = V_hat
            V_prev = V_hat

            oos_rows_this.append({
                "gvkey": gvkey,
                "train_end": train_end,
                "date": pd.Timestamp(d),
                "sigma_hat": float(sigmaV_hat),
                "mu_hat_train": mu_hat,
                "E": E_obs,
                "B_used": B_obs,
                "r": r_obs,
                "V_hat_oos": float(V_hat),
                "dlogV_oos": dlogV,
            })

        if len(oos_rows_this) == 0:
            continue

        oos_df_this = pd.DataFrame(oos_rows_this).sort_values("date").reset_index(drop=True)

        # estimate ONE mu for the OOS quarter (matches your “get mu for next quarter” wording)
        mu_hat_oos = estimate_mu_from_weekly_implied_assets(
            pd.DataFrame({"dlogV": oos_df_this["dlogV_oos"].to_numpy(dtype=float)}),
            float(sigmaV_hat),
            ann_factor=52.0
        )
        mu_hat_oos = float(mu_hat_oos) if np.isfinite(mu_hat_oos) else np.nan
        oos_df_this["mu_hat_oos"] = mu_hat_oos

        # vectorized PDs
        oos_df_this["PD_Q_1y_oos"] = merton_pd_rn_1y(
            oos_df_this["V_hat_oos"].to_numpy(float),
            oos_df_this["B_used"].to_numpy(float),
            oos_df_this["r"].to_numpy(float),
            oos_df_this["sigma_hat"].to_numpy(float),
        )

        oos_df_this["PD_P_1y_oos"] = np.nan
        mask = np.isfinite(oos_df_this["mu_hat_oos"].to_numpy(float))
        if mask.any():
            oos_df_this.loc[mask, "PD_P_1y_oos"] = merton_pd_physical_1y(
                oos_df_this.loc[mask, "V_hat_oos"].to_numpy(float),
                oos_df_this.loc[mask, "B_used"].to_numpy(float),
                oos_df_this.loc[mask, "mu_hat_oos"].to_numpy(float),
                oos_df_this.loc[mask, "sigma_hat"].to_numpy(float),
            )

        roll_weekly_oos_rows.append(oos_df_this)

# Build final rolling outputs
roll_summary_df = pd.DataFrame(roll_summary_rows).sort_values(["train_end","gvkey"]).reset_index(drop=True)

roll_weekly_is_df = (
    pd.concat(roll_weekly_is_rows, ignore_index=True)
      .sort_values(["train_end","gvkey","date"])
      .reset_index(drop=True)
) if len(roll_weekly_is_rows) else pd.DataFrame()

roll_weekly_oos_df = (
    pd.concat(roll_weekly_oos_rows, ignore_index=True)
      .sort_values(["train_end","gvkey","date"])
      .reset_index(drop=True)
) if len(roll_weekly_oos_rows) else pd.DataFrame()

roll_summary_df, roll_weekly_is_df, roll_weekly_oos_df

Rolling windows: 100%|██████████| 43/43 [00:04<00:00,  9.26it/s]


(     gvkey train_start  train_end  oos_start    oos_end    ok msg  sigma_hat  \
 0   100022  2012-04-01 2014-03-31 2014-04-01 2014-06-30  True  ok   0.079366   
 1   100080  2012-04-01 2014-03-31 2014-04-01 2014-06-30  True  ok   0.140146   
 2   100022  2012-07-01 2014-06-30 2014-07-01 2014-09-30  True  ok   0.076176   
 3   100080  2012-07-01 2014-06-30 2014-07-01 2014-09-30  True  ok   0.133382   
 4   100022  2012-10-01 2014-09-30 2014-10-01 2014-12-31  True  ok   0.077528   
 ..     ...         ...        ...        ...        ...   ...  ..        ...   
 81  100080  2022-04-01 2024-03-31 2024-04-01 2024-06-30  True  ok   0.113557   
 82  100022  2022-07-01 2024-06-30 2024-07-01 2024-09-30  True  ok   0.064070   
 83  100080  2022-07-01 2024-06-30 2024-07-01 2024-09-30  True  ok   0.107020   
 84  100022  2022-10-01 2024-09-30 2024-10-01 2024-12-31  True  ok   0.067516   
 85  100080  2022-10-01 2024-09-30 2024-10-01 2024-12-31  True  ok   0.108850   
 
     mu_hat_train         

In [16]:
# 8) Sanity checks + quick previews

print("roll_summary_df shape:", roll_summary_df.shape)
print("roll_weekly_is_df shape:", roll_weekly_is_df.shape)
print("roll_weekly_oos_df shape:", roll_weekly_oos_df.shape)

# Check that we actually have rolling windows
print("Unique train_end windows:", roll_summary_df["train_end"].nunique() if "train_end" in roll_summary_df.columns else None)

# Basic validity checks
if not roll_summary_df.empty:
    print("Share ok:", roll_summary_df["ok"].mean() if "ok" in roll_summary_df.columns else None)
    print("Sigma summary (ok only):")
    print(roll_summary_df.loc[roll_summary_df["ok"] == True, "sigma_hat"].describe())

# Peek one firm across time
if not roll_summary_df.empty:
    g0 = roll_summary_df.loc[roll_summary_df["ok"] == True, "gvkey"].astype(str).unique()
    if len(g0):
        example = g0[0]
        display(
            roll_summary_df[(roll_summary_df["gvkey"] == example) & (roll_summary_df["ok"] == True)]
            [["gvkey","train_end","sigma_hat","mu_hat_train","PD_Q_1y_is","PD_P_1y_is"]]
            .head(15)
        )

# Peek OOS PDs for same firm
if not roll_weekly_oos_df.empty and len(g0):
    display(
        roll_weekly_oos_df[roll_weekly_oos_df["gvkey"] == example]
        [["gvkey","train_end","date","sigma_hat","mu_hat_oos","PD_Q_1y_oos","PD_P_1y_oos"]]
        .head(30)
    )

roll_summary_df shape: (4, 15)
roll_weekly_is_df shape: (420, 9)
roll_weekly_oos_df shape: (56, 13)
Unique train_end windows: 2
Share ok: 1.0
Sigma summary (ok only):
count    4.000000
mean     0.107267
std      0.034196
min      0.076176
25%      0.078568
50%      0.106374
75%      0.135073
max      0.140146
Name: sigma_hat, dtype: float64


,gvkey,train_end,sigma_hat,mu_hat_train,PD_Q_1y_is,PD_P_1y_is
0,100022,2014-03-31,0.079366,0.075953,3.715969e-08,1.280220e-10
2,100022,2014-06-30,0.076176,0.100709,7.867343e-09,1.512027e-12


,gvkey,train_end,date,sigma_hat,mu_hat_oos,PD_Q_1y_oos,PD_P_1y_oos
0,100022,2014-03-31,2014-04-04,0.079366,0.020209,2.037122e-08,5.114488e-09
1,100022,2014-03-31,2014-04-11,0.079366,0.020209,5.412197e-08,1.402631e-08
2,100022,2014-03-31,2014-04-18,0.079366,0.020209,3.262744e-08,8.204367e-09
3,100022,2014-03-31,2014-04-25,0.079366,0.020209,6.415138e-08,1.685841e-08
4,100022,2014-03-31,2014-05-02,0.079366,0.020209,7.424729e-08,1.927174e-08
5,100022,2014-03-31,2014-05-09,0.079366,0.020209,9.465028e-08,2.458105e-08
6,100022,2014-03-31,2014-05-16,0.079366,0.020209,1.478620e-07,3.865973e-08
7,100022,2014-03-31,2014-05-23,0.079366,0.020209,6.000773e-08,1.489456e-08
8,100022,2014-03-31,2014-05-30,0.079366,0.020209,3.343194e-08,8.016448e-09
9,100022,2014-03-31,2014-06-06,0.079366,0.020209,2.713461e-08,6.392688e-09
